In [15]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.utils.validation import check_is_fitted, check_array
from typing import List, Union 
from typing import Optional, List, Union  
import warnings
        
def check_my_estimator(estimator): 
    skipped = ['check_estimators_nan_inf']
    for check in _yield_all_checks(estimator):
        if check.func.__name__ in skipped:
            continue
        check(estimator)  

class SmartOutlierHandler(TransformerMixin, BaseEstimator):
    def __init__(
        self,
        method: str = 'iqr',
        replace_strategy: str = 'median',
        border_quantile: float = 0.01,
        contamination: float = 0.05,
        threshold: Optional[float] = None,
        min_std: float = 1e-3,
        default_value: float = 0,
        verbose: bool = False,
        columns: Optional[List[Union[str, int]]] = None,
        max_components: int = 5,
        min_peak_height_ratio: float = 0.05,
        random_state: Optional[int] = None
    ):

        self.method = method
        self.replace_strategy = replace_strategy
        self.border_quantile = border_quantile
        self.contamination = contamination
        self.threshold = threshold
        self.min_std = min_std
        self.default_value = default_value
        self.verbose = verbose
        self.columns = columns
        self.max_components = max_components
        self.min_peak_height_ratio = min_peak_height_ratio
        self.random_state = random_state
            
    def fit(self, X: Union[pd.DataFrame, np.ndarray], y=None):
        X = check_array(X, ensure_2d=True, ensure_all_finite='allow-nan')
        if np.iscomplexobj(X):
            raise ValueError("Complex data not supported")
    
        def log(msg):
            if self.verbose:
                print(msg)
    
        def prepare_dataframe(X):
            if not isinstance(X, pd.DataFrame):
                return pd.DataFrame(X), True
            return X, False
    
        def determine_columns(X, generated_cols):
            if self.columns is None:
                if generated_cols:
                    return list(range(X.shape[1]))
                return [i for i in range(X.shape[1]) if pd.api.types.is_numeric_dtype(X.iloc[:, i])]
            valid_cols = []
            for col in self.columns:
                if isinstance(col, int) and col < X.shape[1]:
                    valid_cols.append(col)
                elif isinstance(col, str) and col in X.columns:
                    valid_cols.append(X.columns.get_loc(col))
            return valid_cols
    
        def iqr_bounds(series):
            Q1, Q3 = series.quantile([0.25, 0.75])
            IQR = Q3 - Q1
            return Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    
        def quantile_clip_bounds(series):
            return (
                series.quantile(self.border_quantile),
                series.quantile(1 - self.border_quantile)
            )
    
        def isolation_forest_model(series):
            iso = IsolationForest(
                contamination=self.contamination,
                random_state=42
            )
            iso.fit(series.values.reshape(-1, 1))
            return iso
    
        def process_column(series, col_idx, col_name):
            if len(series) < 2 or series.nunique() < 2:
                log(f"\n📌 [Column {col_name}] Skipped - insufficient data (n={len(series)}, unique={series.nunique()})")
                return
    
            log("\n" + "-" * 40)
            log(f"🔧 Processing column: {col_name}")
            log(f"📈 Data stats: mean={series.mean():.2f}, std={series.std():.2f}")
            log(f"📊 Min={series.min():.2f}, 25%={series.quantile(0.25):.2f}, "
                f"50%={series.median():.2f}, 75%={series.quantile(0.75):.2f}, Max={series.max():.2f}")
            log(f"🔢 Non-null values: {len(series)}")
    
            if self.method == 'gmm':
                log("\n🔮 Fitting Gaussian Mixture Model...")
                model, clip_info = self._fit_gmm(series)
                self.clip_params_[col_idx] = {'model': model, 'clip_info': clip_info}
                log(f"✅ GMM fitted with {model.n_components} components")
                for cluster, bounds in clip_info.items():
                    log(f"   Cluster {cluster}: bounds=[{bounds['low']:.2f}, {bounds['high']:.2f}]")
    
            elif self.method == 'iqr':
                low, high = iqr_bounds(series)
                self.clip_params_[col_idx] = {'low': low, 'high': high}
                log("\n📏 Using IQR method:")
                log(f"   Q1: {series.quantile(0.25):.2f}, Q3: {series.quantile(0.75):.2f}")
                log(f"   Lower bound: {low:.2f}")
                log(f"   Upper bound: {high:.2f}")
    
            elif self.method == 'clip':
                low, high = quantile_clip_bounds(series)
                self.clip_params_[col_idx] = {'low': low, 'high': high}
                log("\n✂️ Using quantile clipping:")
                log(f"   Lower bound ({self.border_quantile * 100}%): {low:.2f}")
                log(f"   Upper bound ({(1 - self.border_quantile) * 100}%): {high:.2f}")
    
            elif self.method == 'isolation_forest':
                log("\n🌲 Fitting Isolation Forest...")
                model = isolation_forest_model(series)
                low, high = quantile_clip_bounds(series)
                self.clip_params_[col_idx] = {'model': model, 'low': low, 'high': high}
                log(f"✅ Isolation Forest fitted with contamination={self.contamination}")
                log(f"   Fallback bounds: [{low:.2f}, {high:.2f}]")
    
            if self.method in ['iqr', 'clip', 'isolation_forest']:
                bounds = self.clip_params_[col_idx]
                n_low = (series < bounds['low']).sum()
                n_high = (series > bounds['high']).sum()
                total = n_low + n_high
                pct = total / len(series) * 100
                log(f"🔍 Outliers detected: {total} ({pct:.1f}%)")
                log(f"   - Below lower bound: {n_low}")
                log(f"   - Above upper bound: {n_high}")
    
        # --- Main Execution Flow ---
        X, generated_cols = prepare_dataframe(X)
        self.n_features_in_ = X.shape[1]
        self.feature_names_in_ = np.asarray(X.columns, dtype=object) if not generated_cols else None
        self.columns_ = determine_columns(X, generated_cols)
    
        if not self.columns_:
            raise ValueError("No valid numeric columns found to process")
    
        self.clip_params_ = {}
    
        for col_idx in self.columns_:
            col_name = X.columns[col_idx]
            series = X.iloc[:, col_idx].dropna()
            try:
                process_column(series, col_idx, col_name)
            except Exception as e:
                log(f"\n❌ [Column {col_name}] Error during fitting: {str(e)}")
                continue
    
        log("\n" + "=" * 50)
        log("✅ SmartOutlierHandler fitting completed successfully!")
        log(f"📋 Processed {len(self.clip_params_)} columns")
        log("=" * 50 + "\n")
    
        return self


    def transform(self, X: Union[pd.DataFrame, np.ndarray]) -> np.ndarray:
        """Transform the data by handling outliers with proper dtype preservation."""
        check_is_fitted(self)
    
        X = check_array(X, ensure_2d=True, ensure_all_finite='allow-nan')
        if np.iscomplexobj(X):
            raise ValueError("Complex data not supported")
    
        def log(msg):
            if self.verbose:
                print(msg)
    
        def to_dataframe(X):
            return pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
    
        def validate_shape(X):
            if X.shape[1] != self.n_features_in_:
                raise ValueError(f"X has {X.shape[1]} features, but SmartOutlierHandler expects {self.n_features_in_}.")
    
        def prepare_column_metadata(X):
            cols = [X.columns[col_idx] for col_idx in self.columns_ if col_idx < X.shape[1]]
            log(f"🔢 Processing {len(cols)} numeric columns")
            log(f"📋 Columns: {cols}")
            return cols, {col: X[col].copy() for col in cols}
    
        def detect_and_replace_outliers(X, col_idx, col_name, series):
            nonlocal total_outliers
            mask_notna = series.notna()
            valid_vals = series[mask_notna]
    
            if len(valid_vals) == 0:
                log(f"\n📌 [Column {col_name}] No non-null values to process")
                return
    
            log(f"\n🔧 Processing column: {col_name}")
            log(f"📊 Non-null values: {len(valid_vals)}")
            col_outliers = 0
    
            if self.method == 'gmm':
                log("🔮 Using GMM method for outlier detection")
                clusters = self.clip_params_[col_idx]['model'].predict(valid_vals.values.reshape(-1, 1))
                for idx, cluster_id in zip(valid_vals.index, clusters):
                    val = valid_vals.loc[idx]
                    bounds = self.clip_params_[col_idx]['clip_info'][cluster_id]
                    if val < bounds['low'] or val > bounds['high']:
                        new_val = self._replace(val, valid_vals, bounds['low'], bounds['high'])
                        X.loc[idx, col_name] = new_val
                        col_outliers += 1
    
            elif self.method == 'isolation_forest':
                log("🌲 Using Isolation Forest for outlier detection")
                preds = self.clip_params_[col_idx]['model'].predict(valid_vals.values.reshape(-1, 1))
                outliers_idx = valid_vals.index[preds == -1]
                col_outliers = len(outliers_idx)
                for idx in outliers_idx:
                    val = valid_vals.loc[idx]
                    bounds = self.clip_params_[col_idx]
                    new_val = self._replace(val, valid_vals, bounds['low'], bounds['high'])
                    X.loc[idx, col_name] = new_val
    
            else:
                bounds = self.clip_params_[col_idx]
                log(f"📏 Using {'IQR' if self.method == 'iqr' else 'quantile'} bounds:")
                log(f"   Lower: {bounds['low']:.2f}, Upper: {bounds['high']:.2f}")
                for idx in valid_vals.index:
                    val = valid_vals.loc[idx]
                    if val < bounds['low'] or val > bounds['high']:
                        new_val = self._replace(val, valid_vals, bounds['low'], bounds['high'])
                        X.loc[idx, col_name] = new_val
                        col_outliers += 1
    
            total_outliers += col_outliers
            if self.verbose:
                outlier_pct = col_outliers / len(valid_vals) * 100
                log(f"🔍 Outliers handled: {col_outliers} ({outlier_pct:.1f}%)")
                log(f"🔄 Replacement strategy: {self.replace_strategy}")
    
        def restore_original_dtypes(X, original_values, numeric_cols):
            for col in numeric_cols:
                orig_dtype = original_values[col].dtype
                if pd.api.types.is_integer_dtype(orig_dtype):
                    if X[col].isna().any():
                        new_dtype = f'Int{orig_dtype.itemsize * 8}'
                        log(f"\n🔄 Converting {col} to nullable integer type: {new_dtype}")
                        X[col] = X[col].round().astype(new_dtype)
                    else:
                        log(f"\n🔄 Converting {col} back to original integer type: {orig_dtype}")
                        X[col] = X[col].round().astype(orig_dtype)
                else:
                    X[col] = X[col].astype(orig_dtype)
    
        # --- Main Execution ---
        X = to_dataframe(X)
        validate_shape(X)
        X = X.copy()
        numeric_cols, original_values = prepare_column_metadata(X)
        X[numeric_cols] = X[numeric_cols].astype(np.float64)
    
        total_outliers = 0
        for col_idx in self.clip_params_:
            if col_idx >= X.shape[1]:
                continue
            col_name = X.columns[col_idx]
            series = X[col_name]
            detect_and_replace_outliers(X, col_idx, col_name, series)
    
        restore_original_dtypes(X, original_values, numeric_cols)
    
        log("\n" + "=" * 50)
        log("✅ Transformation completed successfully!")
        log(f"🔍 Total outliers handled: {total_outliers}")
        log("=" * 50 + "\n")
    
        return X.values


    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

    def _fit_gmm(self, series: pd.Series):
        """Fit Gaussian Mixture Model to detect clusters with robust enhancements."""
        data = series.values.reshape(-1, 1)
    
        def log(msg):
            if self.verbose:
                print(msg)
    
        def compute_kde_and_peaks(data_1d):
            log("📈 Computing Kernel Density Estimation...")
            kde = gaussian_kde(data_1d)
            x_vals = np.linspace(data.min(), data.max(), 1000)
            y_vals = kde(x_vals)
            log("🏔️ Detecting peaks in density estimation...")
            peaks, _ = find_peaks(
                y_vals,
                height=self.min_peak_height_ratio * y_vals.max(),
                prominence=0.1 * y_vals.max()
            )
            log(f"🔍 Found {len(peaks)} potential peaks in density")
            return peaks
    
        def select_best_gmm(data, peaks):
            bic_scores = []
            max_possible = min(len(peaks), self.max_components) or 1
            n_components_range = range(1, max_possible + 1)
    
            log("\n🔮 Testing different numbers of components:")
            log(f"🔢 Testing components: {list(n_components_range)}")
    
            for n in n_components_range:
                gmm = GaussianMixture(n_components=n, covariance_type='full', random_state=42, reg_covar=1e-6)
                gmm.fit(data)
                bic = gmm.bic(data)
                bic_scores.append(bic)
                log(f"   - Components={n}: BIC={bic:.2f}")
    
            best_n = n_components_range[np.argmin(bic_scores)]
            best_model = GaussianMixture(n_components=best_n, covariance_type='full', random_state=42, reg_covar=1e-6)
            best_model.fit(data)
            log(f"\n🏆 Selected model with {best_n} components (lowest BIC)")
            log(f"📊 Component weights: {best_model.weights_}")
            return best_model
    
        def compute_cluster_bounds(gmm, data):
            clusters = gmm.predict(data)
            bounds = {}
            valid = 0
            log("\n🔍 Validating clusters:")
    
            for c in range(gmm.n_components):
                cluster_data = data[clusters == c]
                cluster_size = len(cluster_data)
    
                if cluster_size > max(5, 0.05 * len(data)):
                    low = np.quantile(cluster_data, self.border_quantile)
                    high = np.quantile(cluster_data, 1 - self.border_quantile)
                    bounds[c] = {'low': low, 'high': high}
                    valid += 1
                    log(f"   ✅ Cluster {c}: size={cluster_size} ({cluster_size / len(data) * 100:.1f}%)")
                    log(f"      Bounds: [{low:.2f}, {high:.2f}]")
                else:
                    log(f"   ❌ Cluster {c}: too small (size={cluster_size}), skipping")
    
            if valid == 0:
                log("⚠️ No valid clusters found - using global quantiles")
                low = np.quantile(data, self.border_quantile)
                high = np.quantile(data, 1 - self.border_quantile)
                bounds[0] = {'low': low, 'high': high}
                log(f"   Global bounds: [{low:.2f}, {high:.2f}]")
    
            return bounds
    
        log("\n🔍 Starting GMM fitting process...")
        log(f"📊 Data range: [{data.min():.2f}, {data.max():.2f}]")
        
        peaks = compute_kde_and_peaks(data.ravel())
        best_gmm = select_best_gmm(data, peaks)
        cluster_clip = compute_cluster_bounds(best_gmm, data)
    
        return best_gmm, cluster_clip


    def _replace(self, value: float, series: pd.Series, 
                low: float, high: float) -> float:
        """Replace outlier value according to strategy."""
        #if self.verbose: print(f"   🔄 Replacing outlier value {value:.2f} (bounds: [{low:.2f}, {high:.2f}])")
        
        if self.replace_strategy == 'clip':
            new_val = np.clip(value, low, high)
            #if self.verbose: print(f"      ✂️ Clipped to: {new_val:.2f}")
            return new_val
        elif self.replace_strategy == 'null':
            #if self.verbose: print("      🚫 Replaced with NaN")
            return np.nan
        elif self.replace_strategy == 'mean':
            new_val = series.mean()
            #if self.verbose: print(f"      📊 Replaced with mean: {new_val:.2f}")
            return new_val
        elif self.replace_strategy == 'median':
            new_val = series.median()
            #if self.verbose: print(f"      📊 Replaced with median: {new_val:.2f}")
            return new_val
        else:
            raise ValueError(f"Unknown replace strategy: {self.replace_strategy}")

    def get_feature_names_out(self, input_features=None):
        """ Get output feature names. """
        check_is_fitted(self)
        if input_features is None:
            if hasattr(self, 'feature_names_in_'):
                return self.feature_names_in_
            return np.array([f"feature_{i}" for i in range(self.n_features_in_)])
        return np.asarray(input_features, dtype=object)

    @classmethod
    def build_full_pipeline(cls, data: pd.DataFrame, 
                           numeric_cols: List[str] = None,
                           method: str = 'iqr',
                           replace_strategy: str = 'clip',
                           **kwargs) -> Pipeline:
       
       
        if numeric_cols is None:
            numeric_cols = data.select_dtypes(include=np.number).columns.tolist()
        
        return Pipeline([
            ('preprocessor', ColumnTransformer([
                ('numeric', Pipeline([
                    ('imputer', SimpleImputer(strategy='median')),
                    ('outlier', cls(
                        columns=numeric_cols,
                        method=method,
                        replace_strategy=replace_strategy,
                        **kwargs
                    ))
                ]), numeric_cols),
                ('passthrough', 'passthrough', 
                 list(set(data.columns) - set(numeric_cols)))
            ], remainder='passthrough'))
        ])
 

In [16]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from time import time
 
X, y = make_classification(
    n_samples=800,
    n_features=15,
    n_informative=10,
    n_redundant=2,
    n_classes=2,
    random_state=42,
    flip_y=0.15,  
    class_sep=0.8  
)
 
rng = np.random.RandomState(42)
for i in range(3):
    outlier_mask = rng.rand(X.shape[0]) < 0.07
    X[outlier_mask, i] = X[outlier_mask, i] * 15 + rng.normal(10, 2, size=outlier_mask.sum())

# Convert to DataFrame
X_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, test_size=0.2, random_state=42, stratify=y
)

# Optimized pipeline
pipeline = Pipeline([
    ('outlier', SmartOutlierHandler(verbose=True)),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
])

# Focused parameter grid
param_grid = {
    'outlier__method': ['iqr', 'isolation_forest', 'gmm'],
    'outlier__replace_strategy': ['clip', 'median', 'mean', 'null'],
    #'outlier__contamination': [0.05, 0.1],
    #'classifier__n_estimators': [50, 100],
    #'classifier__max_depth': [5, 10, None]
}

# Efficient GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)

# Run with timing
start_time = time()
grid_search.fit(X_train, y_train)
print(f"\nGrid search completed in {time()-start_time:.2f} seconds")

# Evaluation
print("\nBest parameters:")
print(grid_search.best_params_)
print(f"Best CV accuracy: {grid_search.best_score_:.4f}")

# Proper test evaluation
from sklearn.metrics import classification_report
y_pred = grid_search.best_estimator_.predict(X_test)
print("\nTest set performance:")
print(classification_report(y_test, y_pred))

# Feature importance analysis
if hasattr(grid_search.best_estimator_['classifier'], 'feature_importances_'):
    importances = grid_search.best_estimator_['classifier'].feature_importances_
    print("\nFeature importances:")
    print(pd.Series(importances, index=X_df.columns).sort_values(ascending=False))

Fitting 3 folds for each of 12 candidates, totalling 36 fits

----------------------------------------
🔧 Processing column: 0
📈 Data stats: mean=1.14, std=9.11
📊 Min=-61.28, 25%=-1.09, 50%=0.30, 75%=1.90, Max=91.87
🔢 Non-null values: 427

📏 Using IQR method:
   Q1: -1.09, Q3: 1.90
   Lower bound: -5.57
   Upper bound: 6.39
🔍 Outliers detected: 28 (6.6%)
   - Below lower bound: 7
   - Above upper bound: 21

----------------------------------------
🔧 Processing column: 1
📈 Data stats: mean=0.78, std=5.57
📊 Min=-21.59, 25%=-0.64, 50%=0.07, 75%=0.81, Max=42.51
🔢 Non-null values: 427

📏 Using IQR method:
   Q1: -0.64, Q3: 0.81
   Lower bound: -2.81
   Upper bound: 2.98
🔍 Outliers detected: 29 (6.8%)
   - Below lower bound: 8
   - Above upper bound: 21

----------------------------------------
🔧 Processing column: 2
📈 Data stats: mean=0.51, std=4.84
📊 Min=-50.72, 25%=-0.66, 50%=0.08, 75%=0.78, Max=35.14
🔢 Non-null values: 427

📏 Using IQR method:
   Q1: -0.66, Q3: 0.78
   Lower bound: -2.83


In [ ]:
pipeline.build_full_pipeline()